# Bootstrap multiple comparisons tutorial

This Jupyter _Python 3_ notebook has been written to accompany the WSC18 paper:

**PRACTICAL CONSIDERATIONS IN SELECTING THE BEST SET OF SIMULATED SYSTEMS**  _by Christine Currie and Tom Monks_.

The notebook provides a worked example of using BootComp to conduct a 2 stage screening and search of a simulation model.  

## 1. Preamble

### 1.1. Detail of the simulation model

The simulation model was used in a 2017 project in the UK to help a hospital, a community healthcare provider and a clinical commissioning group design and plan a new community rehabilitation ward.  In the UK, patients who require rehabilitation are often stuck in a queuing system where there must wait (inappropriately) in a acute hospital bed for a space in the rehabilitaiton ward.  The model investigated the sizing of the new ward in order to minimise patient waiting time whilst meeting probabilitic constraints regarding ward occupancy (bed utilization) and the number of transfers between single sex bays.

<img src="images/DToC.jpg" alt="Delayed Transfers of Care Model" title="Simulation Model and KPIs" />

### 1.2. Output data

The output data for the example analysis are bundled with git repository.  There are three .csv files in the data/ directory for 'waiting times', 'utilization' and 'transfers'.  

The model itself is not needed.  There are 50 replications of 1151 competing designs points.  Users can vary the number of replications used in the two stage procedure.  

The experimental design is also included for reference.

## 2. Prerequisites

### 2.1. BootComp Modules

In [1]:
import Bootstrap as bs
import BootIO as io
import ConvFuncs as cf

In [2]:
#DEV
import Bootstrap_crn as crn

### 2.2. Python Data Science Modules

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 3. Procedure: Stage 1

### Step 1: Read in initial $ n_1 $  replications

In [4]:
N_BOOTS = 2000
n_1 = 5
INPUT_DATA1 = "data/replications_wait_times.csv"
INPUT_DATA2 = "data/replications_util.csv"
INPUT_DATA3 = "data/replications_transfers.csv"
DESIGN = "data/doe.csv"

In [5]:
system_data_wait = crn.load_scenarios(INPUT_DATA1, exclude_reps = 50-n_1)
system_data_util = crn.load_scenarios(INPUT_DATA2, exclude_reps = 50-n_1)
system_data_tran = crn.load_scenarios(INPUT_DATA3, exclude_reps = 50-n_1)

N_SCENARIOS = system_data_wait.shape[1]
N_REPS = system_data_wait.shape[0]

print("Loaded waiting time data. {0} systems; {1} replications".format(system_data_wait.shape[1], system_data_wait.shape[0]))
print("Loaded utilzation data. {0} systems; {1} replications".format(system_data_util.shape[1], system_data_util.shape[0]))
print("Loaded transfers data. {0} systems; {1} replications".format(system_data_tran.shape[1], system_data_tran.shape[0]))

Loaded waiting time data. 1051 systems; 5 replications
Loaded utilzation data. 1051 systems; 5 replications
Loaded transfers data. 1051 systems; 5 replications


In [6]:
df_tran = pd.DataFrame(system_data_tran)
df_util = pd.DataFrame(system_data_util)
df_wait = pd.DataFrame(system_data_wait)

### Step 2: Limit to systems that satisfy chance constraints

Bootstrap function arguments

In [7]:
N_BOOTS = 1000
args =  bs.BootstrapArguments()

args.nboots = N_BOOTS
args.nscenarios = N_SCENARIOS
args.point_estimate_func = bs.bootstrap_mean


In [8]:
def bootstrap_chance_constraint(data, threshold, boot_args, p=0.95, kind='lower'):
    """
    Bootstrap a chance constraint for k systems and filter out systems 
    where p% of resamples are greater a threshold t.  
    
    Example 1. A lower limit.  If the chance constaint was related to utilization it could be stated as 
    filter out any systems where 95% of the distribution is greater than 80%.
    
    Example 2. An upper limit.  If the chance constraint related to unwanted ward transfers it could be stated 
    as filter out any systems where 95% of the distribution is less than 50 transfers per annum.
    
    Returns a pandas.Series containing of the feasible systems i.e. that do not violate the chance constraint.
    
    @data - a numpy array of the data to bootstrap
    @threshold - the threshold of the chance constraint
    @boot_args - the bootstrap setup class
    @p - the probability cut of for the chance constraint  (default p = 0.95)
    @kind - 'lower' = a lower limit threshold; 'upper' = an upper limit threshold (default = 'lower')
    
    """
    
    valid_operations = ['upper', 'lower']
    
    if kind.lower() not in valid_operations:
        raise ValueError('Parameter @kind must be either set to lower or upper')
    
    resample_list = bs.resample_all_scenarios(data.tolist(), boot_args)
    df_boots = cf.resamples_to_df(resample_list, boot_args.nboots)
    
    if('lower' == kind.lower()):
        
        df_counts = pd.DataFrame(df_boots[df_boots >= threshold].count(), columns = {'count'})
    else:
        df_counts = pd.DataFrame(df_boots[df_boots <= threshold].count(), columns = {'count'})
        
    df_counts['prop'] = df_counts['count'] / boot_args.nboots
    df_counts['pass'] = np.where(df_counts['prop'] >= p, 1, 0)
    df_counts.index -= 1
    
    return df_counts.loc[df_counts['pass'] == 1].index
    
    
    

#### Chance constraint 1:  Utilisation Threshold (value for money)

In [9]:
min_util = 80

In [10]:
p = 0.80

In [11]:
passed_1 = bootstrap_chance_constraint(data = system_data_util.T, threshold=min_util, boot_args=args, p=p)

#### Chance constraint 2: Upper bound on transfers between bays

In [12]:
max_tran = 50

In [13]:
passed_2 = bootstrap_chance_constraint(data = system_data_tran.T, threshold=max_tran, boot_args=args, p=p, kind='upper')

#### Filter for systems that meet all chance constraints

In [14]:
subset = np.intersect1d(passed_1, passed_2)
subset

array([  0,   1,   2,  14,  35,  40,  50,  58,  62,  63,  64,  65,  66,
        67,  68,  79,  88,  89,  96, 101, 116, 119, 129, 130, 131, 132,
       133, 134, 135, 136, 147, 148, 149, 157, 158, 165, 166, 171, 184,
       187, 190, 198, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210,
       219, 220, 221, 222, 229, 230, 231, 237, 238, 244, 249, 256, 259,
       262, 270, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283,
       284, 285, 292, 293, 294, 295, 296, 302, 303, 304, 305, 310, 311,
       312, 316, 317, 318, 322, 323, 326, 328, 329, 330, 332, 333, 335,
       343, 345, 346, 347, 348, 349, 350, 351, 429], dtype=int64)

In [15]:
ind = [1, 2, 3, 4]
df_wait[subset]

,0,1,2,14,35,40,50,58,62,63,...,335,343,345,346,347,348,349,350,351,429
0,0.310817,0.310817,0.310817,0.310817,3.163141,3.144300,17.431210,3.144300,4.445242,10.594455,...,4.087298,0.665171,0.105251,0.189408,0.328546,0.685368,1.559246,4.087298,10.544450,1.558797
1,1.113904,1.113904,1.113904,1.113904,4.945780,4.023624,12.517940,4.023624,4.922641,8.298630,...,4.415987,0.753397,0.388211,0.525971,0.835140,1.360659,2.456865,4.415987,7.959398,2.439476
2,1.631624,1.631624,1.631624,1.631624,15.382675,4.701431,8.429396,4.701431,15.448503,23.046322,...,6.101747,1.150213,0.772979,1.234868,2.124249,4.077171,7.822539,14.966652,22.637512,7.772688
3,3.061275,3.061275,3.061275,3.061275,16.087584,6.952027,21.067490,6.952027,10.441470,15.879639,...,10.071298,1.330142,1.238859,1.666146,2.054018,2.828181,4.324821,10.071298,15.779303,4.317652
4,0.765249,0.765249,0.765249,0.765249,8.666150,2.161422,6.125000,2.161422,8.689823,13.705802,...,8.356543,0.365550,0.271692,0.515034,1.028136,2.512694,5.461421,8.356543,14.039629,5.460331


In [16]:
#need to zero index.   - DO I ACTUALLY NEED TO DO THIS?
subset_zero = [x - 1 for x in subset]
subset_zero = subset  ##NOTE MAY NEED TO REOMVE.
subset_waits = df_wait[subset_zero].mean()
subset_waits.rename('wait', inplace=True)
subset_utils = df_util[subset_zero].mean()
subset_utils.rename('util', inplace=True)
subset_tran = df_tran[subset_zero].mean()
subset_tran.rename('tran', inplace=True)

0       0.2
1      28.2
2      37.4
14     36.0
35     32.2
40     41.2
50     23.8
58     41.2
62     35.2
63     15.4
64      8.8
65      2.0
66      0.0
67     26.0
68     40.2
79     26.0
88      0.0
89     33.6
96     42.0
101    31.8
116    30.2
119     0.0
129    37.0
130    21.4
131    10.8
132     3.0
133     0.0
134     0.0
135    23.2
136    35.4
       ... 
295    23.0
296    31.0
302    24.2
303    12.6
304    21.6
305    31.6
310    22.4
311    16.6
312    26.6
316    36.8
317    20.4
318    31.2
322    27.4
323    38.8
326    30.0
328    40.2
329    26.4
330    33.2
332    21.8
333    37.4
335     5.8
343    28.4
345    38.0
346    28.8
347    18.6
348    14.2
349     5.8
350     2.2
351     0.6
429     2.8
Name: tran, dtype: float64

List and rank the systems along with their peformance measures

In [17]:
subset_kpi = pd.concat([subset_waits, subset_utils, subset_tran], axis=1)

In [18]:
subset_kpi.sort_values(by=['wait', 'util', 'tran'])

,wait,util,tran
279,0.256866,82.502992,0.0
293,0.256866,82.502992,10.6
280,0.256866,82.502992,12.2
303,0.256866,82.502992,12.6
281,0.256866,82.502992,15.6
311,0.256866,82.502992,16.6
294,0.256866,82.502992,18.4
282,0.256866,82.502992,20.4
317,0.256866,82.502992,20.4
304,0.256866,82.502992,21.6


In [19]:
best_system_index = subset_kpi.sort_values(by=['wait', 'util', 'tran']).index[0]

In [20]:
best_system_index

279

### Step 3: setup differences

In [21]:
feasible_systems = df_wait[subset_zero]

In [22]:
diffs =  pd.DataFrame(feasible_systems.as_matrix().T - np.array(feasible_systems[best_system_index])).T
diffs.columns = subset

### Step 4: Bootstrap differences

In [23]:
resample_diffs = bs.resample_all_scenarios(diffs.values.T.tolist(), args)

In [24]:
df_boots_diffs= cf.resamples_to_df(resample_diffs, N_BOOTS)
df_boots_diffs.columns = subset
df_boots_diffs.shape

(1000, 113)

### Step 5: Rank systems

y% of bootstraps are within x% of the mean

In [25]:
x = 0.1
y = 0.95

In [26]:
indifference = feasible_systems[best_system_index].mean() * x
indifference

0.025686578680000002

In [27]:
#convert numbers to 0 or 1
# 1 = difference less than 0.244
# 0 = difference greater than 0.244

def indifferent(x, indifference):
    """
    
    """
    if x <= indifference:
        return 1
    else:
        return 0

In [28]:
df_indifference = df_boots_diffs.applymap(lambda x: indifferent(x, indifference))
df_indifference

,0,1,2,14,35,40,50,58,62,63,...,335,343,345,346,347,348,349,350,351,429
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
threshold = N_BOOTS * y
df_within_limit = df_indifference.sum(0)
df_within_limit= pd.DataFrame(df_within_limit, columns=['sum'])
take_forward = df_within_limit.loc[df_within_limit['sum'] >= threshold].index

In [30]:
take_forward

Int64Index([279, 280, 281, 282, 283, 284, 285, 293, 294, 295, 296, 303, 304,
            305, 311, 312, 317, 318, 322, 323, 326, 330, 333],
           dtype='int64')

## 4. Procedure - Stage 2

### Step 6: More replicates of promcing solutions.

User simulates $ n_2 $ additional replicates for the feasible solutions brought forward from stage 1.

Example = 50 replicates (45 extra)

In [31]:
df_wait_s2 = pd.DataFrame(crn.load_scenarios(INPUT_DATA1))[take_forward]
df_util_s2 = pd.DataFrame(crn.load_scenarios(INPUT_DATA2))[take_forward]
df_tran_s2 = pd.DataFrame(crn.load_scenarios(INPUT_DATA3))[take_forward]

N_SCENARIOS = df_wait_s2.shape[1]
N_REPS = df_wait_s2.shape[0]

print("Loaded waiting time data. {0} systems; {1} replications".format(df_wait_s2.shape[1], df_wait_s2.shape[0]))
print("Loaded utilzation data. {0} systems; {1} replications".format(df_util_s2.shape[1], df_util_s2.shape[0]))
print("Loaded transfers data. {0} systems; {1} replications".format(df_tran_s2.shape[1], df_tran_s2.shape[0]))

Loaded waiting time data. 23 systems; 50 replications
Loaded utilzation data. 23 systems; 50 replications
Loaded transfers data. 23 systems; 50 replications


### Step 7: Repeat steps 2 - 5

#### Step 2 - Chance contraints

In [32]:
passed_1 = bootstrap_chance_constraint(data = df_util_s2.values.T, threshold=min_util, boot_args=args)

In [33]:
passed_1

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22],
           dtype='int64')

In [34]:
take_forward

Int64Index([279, 280, 281, 282, 283, 284, 285, 293, 294, 295, 296, 303, 304,
            305, 311, 312, 317, 318, 322, 323, 326, 330, 333],
           dtype='int64')

In [35]:
cc_1 = np.array([take_forward[x-1] for x in passed_1])
cc_1

array([333, 279, 280, 281, 282, 283, 284, 285, 293, 294, 295, 296, 303,
       304, 305, 311, 312, 317, 318, 322, 323, 326, 330], dtype=int64)

In [36]:
passed_2 = bootstrap_chance_constraint(data = df_tran_s2.values.T, threshold=max_tran, boot_args=args, kind='upper')

In [37]:
passed_2

Int64Index([0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20,
            21],
           dtype='int64')

In [38]:
cc_2 = np.array([take_forward[x-1] for x in passed_2])
cc_2

array([333, 279, 280, 281, 282, 283, 285, 293, 294, 295, 296, 303, 304,
       305, 311, 312, 317, 318, 323, 326], dtype=int64)

In [39]:
subset = np.intersect1d(cc_1, cc_2)
subset

array([279, 280, 281, 282, 283, 285, 293, 294, 295, 296, 303, 304, 305,
       311, 312, 317, 318, 323, 326, 333], dtype=int64)

In [60]:
def get_subset_kpi(subset):
    subset_waits = df_wait_s2[subset].mean()
    subset_waits.rename('wait', inplace=True)
    subset_utils = df_util_s2[subset].mean()
    subset_utils.rename('util', inplace=True)
    subset_tran = df_tran_s2[subset].mean()
    subset_tran.rename('tran', inplace=True)
    
    subset_kpi = pd.concat([subset_waits, subset_utils, subset_tran], axis=1)
    subset_kpi.index.rename('System', inplace=True)
    
    return subset_kpi

In [40]:

subset_waits = df_wait_s2[subset].mean()
subset_waits.rename('wait', inplace=True)
subset_utils = df_util_s2[subset].mean()
subset_utils.rename('util', inplace=True)
subset_tran = df_tran_s2[subset].mean()
subset_tran.rename('tran', inplace=True)

279     0.00
280    16.06
281    21.60
282    27.54
283    33.78
285    46.72
293    15.82
294    22.66
295    29.74
296    38.98
303    20.30
304    28.84
305    39.16
311    24.58
312    34.74
317    28.78
318    41.10
323    47.16
326    38.64
333    47.56
Name: tran, dtype: float64

In [41]:
subset_kpi = pd.concat([subset_waits, subset_utils, subset_tran], axis=1)
subset_kpi.index.rename('System', inplace=True)

In [42]:
subset_kpi.sort_values(by=['wait', 'util', 'tran'])


,wait,util,tran
System,,,
317,0.299512,82.729884,28.78
279,0.299560,82.729884,0.00
293,0.299560,82.729884,15.82
280,0.299560,82.729884,16.06
303,0.299560,82.729884,20.30
281,0.299560,82.729884,21.60
294,0.299560,82.729884,22.66
311,0.299560,82.729884,24.58
282,0.299560,82.729884,27.54


In [43]:
best_system_index = subset_kpi.sort_values(by=['wait', 'util', 'tran']).index[0]

In [44]:
best_system_index

317

### Step [?]  Setup differences from best (stage 2)

In [45]:
feasible_systems = df_wait_s2[subset]
diffs =  pd.DataFrame(feasible_systems.as_matrix().T - np.array(feasible_systems[best_system_index])).T
diffs.columns = subset

### Bootstrap differences

In [46]:
resample_diffs = bs.resample_all_scenarios(diffs.values.T.tolist(), args)


In [47]:
df_boots_diffs= cf.resamples_to_df(resample_diffs, args.nboots)
df_boots_diffs.columns = subset
df_boots_diffs.shape

(1000, 20)

In [48]:
x = 0.1
y = 0.95

In [49]:
indifference = feasible_systems[best_system_index].mean() * x
indifference

0.029951178340000013

In [50]:
df_indifference = df_boots_diffs.applymap(lambda x: indifferent(x, indifference))
df_indifference

,279,280,281,282,283,285,293,294,295,296,303,304,305,311,312,317,318,323,326,333
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
7,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
10,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1


In [51]:
threshold = args.nboots * y
df_within_limit = df_indifference.sum(0)
df_within_limit= pd.DataFrame(df_within_limit, columns=['sum'])
final_set = df_within_limit.loc[df_within_limit['sum'] >= threshold].index

In [52]:
final_set

Int64Index([279, 280, 281, 282, 283, 285, 293, 294, 295, 296, 303, 304, 305,
            311, 312, 317, 318, 326, 333],
           dtype='int64')

Final set of feasible systems selected from the competing designs

In [53]:
df_doe = pd.read_csv(DESIGN, index_col='System')
df_doe.index -= 1
#subtract 1 from index so taht it matches zero indexing in analysis.



In [61]:
subset_kpi = get_subset_kpi(final_set)
subset_kpi

,wait,util,tran
System,,,
279,0.299560,82.729884,0.00
280,0.299560,82.729884,16.06
281,0.299560,82.729884,21.60
282,0.299560,82.729884,27.54
283,0.299624,82.729884,33.78
285,0.300516,82.730467,46.72
293,0.299560,82.729884,15.82
294,0.299560,82.729884,22.66
295,0.299564,82.729884,29.74


In [63]:
temp = df_doe[df_doe.index.isin(final_set)]
#subset_kpi = subset_kpi.applymap(lambda x: '%.4f' % x)
df_final = pd.concat([temp, subset_kpi], axis=1)
df_final.sort_values(by=['wait', 'util', 'tran'])


,Total beds,Size of Bays,Number of Bays,Number of Singles,wait,util,tran
System,,,,,,,
317,47,7,2,33,0.299512,82.729884,28.78
279,47,0,0,47,0.299560,82.729884,0.00
293,47,4,2,39,0.299560,82.729884,15.82
280,47,3,3,38,0.299560,82.729884,16.06
303,47,5,2,37,0.299560,82.729884,20.30
281,47,3,4,35,0.299560,82.729884,21.60
294,47,4,3,35,0.299560,82.729884,22.66
311,47,6,2,35,0.299560,82.729884,24.58
282,47,3,5,32,0.299560,82.729884,27.54
